In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

strava_client = stravalib.client.Client(access_token=access_token)
athlete = strava_client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [3]:
## Set up google sheets client, open worksheet
import pygsheets

gc = pygsheets.authorize(outh_file='client_secret.json', no_cache=True)

# Open spreadsheet and then workseet
sh = gc.open('Milburn Ultimate Scores')

In [4]:
## Get last entry from Raw Point Spreadsheet

wks = sh.worksheet_by_title('raw_points')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in wks.get_col(2) if d != '' and d != 'Start Time']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]

# lap_start_date = lap_start_date - timedelta(days=5)

lap_start_date

datetime.datetime(2018, 3, 24, 13, 17, 30)

In [6]:
def get_strava_description(activity, p=False):
    new_activity = strava_client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    try:
        color = color.lower()
    except AttributeError:
        color = None
        
    if p:
        print(new_activity.start_date)
        print(new_activity.description)
        print(team_score, opponent_score, color)
        
    return team_score, opponent_score, color

In [7]:
## Functions
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        try:
            lap_nums.append(int(l['name'].split(' ')[-1]))
            start_times.append(datetime.strptime(l['start_date_local'], '%Y-%m-%dT%H:%M:%SZ'))
            elapsed_times.append(timedelta(seconds=l['elapsed_time']))
        except TypeError:
            lap_nums.append(int(l.name.split(' ')[-1]))
            start_times.append(l.start_date_local)
            elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [8]:
## Compile raw points for export
all_data = []
for run in reversed(runs):
   
    events = extract_events(run)

    point_df = pd.DataFrame(events, columns=['count', 'start_time', 'elapsed_time'])
    point_df['start_time'] = point_df['start_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    point_df['elapsed_time'] = point_df['elapsed_time'].apply(lambda x: x.seconds)
    
    data = point_df.sort_index(ascending=False).as_matrix().tolist()
    all_data = all_data + data
    
    
    

In [52]:
## Write raw points to gsheets
wks = sh.worksheet_by_title('raw_points')
wks.insert_rows(2, values=all_data, number=len(all_data))

In [9]:
## Get last entry from Data Spreadsheet

wks = sh.worksheet_by_title('game_summaries')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in wks.get_col(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]
lap_start_date
# from datetime import timedelta
# lap_start_date = lap_start_date - timedelta(days=2)

datetime.datetime(2018, 3, 22, 0, 0)

In [29]:
wks = sh.worksheet_by_title('raw_points')
col_names = wks.get_all_values()[1][0:3]

val_lists = wks.get_all_values()[2:]
val_lists = [v[0:3] for v in val_lists]
processed_raw_points = pd.DataFrame(val_lists, columns=col_names)

# Process
processed_raw_points['Type'] = processed_raw_points['Type'].apply(lambda x: int(x))

processed_raw_points['Start Time'] = processed_raw_points['Start Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
processed_raw_points['Day'] = processed_raw_points['Start Time'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))
processed_raw_points['Elapsed Time (Sec)'] = processed_raw_points['Elapsed Time (Sec)'].apply(lambda x: timedelta(seconds=int(x)))


processed_raw_points = processed_raw_points[processed_raw_points['Start Time'] > lap_start_date].sort_values('Start Time', ascending=True).reset_index()
processed_raw_points

,index,Type,Start Time,Elapsed Time (Sec),Day
0,26,2,2018-03-23 12:16:40,00:00:14,2018-03-23
1,25,2,2018-03-23 12:16:56,00:02:57,2018-03-23
2,24,3,2018-03-23 12:19:58,00:02:14,2018-03-23
3,23,1,2018-03-23 12:22:11,00:02:17,2018-03-23
4,22,3,2018-03-23 12:24:29,00:01:37,2018-03-23
5,21,2,2018-03-23 12:26:09,00:01:12,2018-03-23
6,20,1,2018-03-23 12:27:23,00:02:17,2018-03-23
7,19,3,2018-03-23 12:29:53,00:01:25,2018-03-23
8,18,4,2018-03-23 12:31:16,00:00:23,2018-03-23
9,17,2,2018-03-23 12:31:37,00:01:07,2018-03-23


In [30]:
games = []

runs = []
for activity in strava_client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        a = strava_client.get_activity(activity.id)
        runs.append(a)
        
for day, pdf in processed_raw_points.groupby('Day', sort=False):
    print(day)

    
    team_wins, opponent_wins, color = get_strava_description(runs[0], p=True)

    events = pdf[['Type', 'Start Time', 'Elapsed Time (Sec)']].as_matrix().tolist()
    
    for e in events:
        print(e)
    current_games = process_events(events)
    
    for g in current_games:
        g['my_color'] = color
        g['team_wins'] = team_wins
        g['opponent_wins'] = opponent_wins
        

    games = games + current_games


2018-03-23 00:00:00
2018-03-23 17:16:40+00:00
3-0 colors
3 0 colors
[2, Timestamp('2018-03-23 12:16:40'), Timedelta('0 days 00:00:14')]
[2, Timestamp('2018-03-23 12:16:56'), Timedelta('0 days 00:02:57')]
[3, Timestamp('2018-03-23 12:19:58'), Timedelta('0 days 00:02:14')]
[1, Timestamp('2018-03-23 12:22:11'), Timedelta('0 days 00:02:17')]
[3, Timestamp('2018-03-23 12:24:29'), Timedelta('0 days 00:01:37')]
[2, Timestamp('2018-03-23 12:26:09'), Timedelta('0 days 00:01:12')]
[1, Timestamp('2018-03-23 12:27:23'), Timedelta('0 days 00:02:17')]
[3, Timestamp('2018-03-23 12:29:53'), Timedelta('0 days 00:01:25')]
[4, Timestamp('2018-03-23 12:31:16'), Timedelta('0 days 00:00:23')]
[2, Timestamp('2018-03-23 12:31:37'), Timedelta('0 days 00:01:07')]
[3, Timestamp('2018-03-23 12:32:43'), Timedelta('0 days 00:01:18')]
[1, Timestamp('2018-03-23 12:34:05'), Timedelta('0 days 00:03:25')]
[3, Timestamp('2018-03-23 12:37:36'), Timedelta('0 days 00:02:51')]
[3, Timestamp('2018-03-23 12:40:27'), Timedelta(

In [33]:
df = pd.DataFrame(games).dropna()
df

,end_time,events,game_num,my_color,my_point,opponent_point,opponent_wins,start_time,team_point,team_wins,win
0,2018-03-23 12:31:39,"[(opponent_point, 2018-03-23 12:16:40, 0 days ...",0,colors,3,3,0,2018-03-23 12:16:40,5,3,True
1,2018-03-23 12:46:51,"[(opponent_point, 2018-03-23 12:31:37, 0 days ...",1,colors,3,2,0,2018-03-23 12:31:37,5,3,True
2,2018-03-23 13:17:41,"[(opponent_point, 2018-03-23 12:46:59, 0 days ...",2,colors,1,4,0,2018-03-23 12:46:59,5,3,True


In [36]:
df['date'] = df.end_time.apply(lambda x: datetime(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2018-03-23 0        2018-03-23 12:31:39   
           1        2018-03-23 12:46:51   
           2        2018-03-23 13:17:41   

                                                                events  \
date       game_num                                                      
2018-03-23 0         [(opponent_point, 2018-03-23 12:16:40, 0 days ...   
           1         [(opponent_point, 2018-03-23 12:31:37, 0 days ...   
           2         [(opponent_point, 2018-03-23 12:46:59, 0 days ...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-23 0                0   colors         3               3   
           1                1   colors         3               2   
           2                2   colors         1               4   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-23 0                    0 2018-03-23 12:16:40           5         3   
           1                    0 2018-03-23 12:31:37           5         3   
           2                    0 2018-03-23 12:46:59           5         3   

                      win       date  
date       game_num                   
2018-03-23 0         True 2018-03-23  
           1         True 2018-03-23  
           2         True 2018-03-23

In [37]:
pdf = df

In [38]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [39]:
for (date, game_num), row in pdf.iterrows():      
    
    if row.my_color == 'white':
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'opponent_wins']            
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'opponent_wins']    
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [40]:
pdf

end_time  \
date       game_num                       
2018-03-23 0        2018-03-23 12:31:39   
           1        2018-03-23 12:46:51   
           2        2018-03-23 13:17:41   

                                                                events  \
date       game_num                                                      
2018-03-23 0         [(opponent_point, 2018-03-23 12:16:40, 0 days ...   
           1         [(opponent_point, 2018-03-23 12:31:37, 0 days ...   
           2         [(opponent_point, 2018-03-23 12:46:59, 0 days ...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-23 0                0   colors         3               3   
           1                1   colors         3               2   
           2                2   colors         1               4   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-23 0                    0 2018-03-23 12:16:40           5         3   
           1                    0 2018-03-23 12:31:37           5         3   
           2                    0 2018-03-23 12:46:59           5         3   

                      win       date white_wins color_wins white_point  \
date       game_num                                                      
2018-03-23 0         True 2018-03-23          0          3           3   
           1         True 2018-03-23          0          3           2   
           2         True 2018-03-23          0          3           4   

                    color_point game_winner  
date       game_num                          
2018-03-23 0                  5       Color  
           1                  5       Color  
           2                  5       Color

In [41]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [42]:
scores = []
for (date, game_num), game in pdf.iterrows():
    base_dict = dict(date=date, end_time=game.end_time, game_num=game_num, white_wins=game.white_wins, color_wins=game.color_wins, game_winner=game.game_winner)    
    scores.append(merge_two_dicts(base_dict, dict(team='white', team_score=game.white_point, my_score=game.my_point if game.my_color == 'white' else None)))
    scores.append(merge_two_dicts(base_dict, dict(team='color', team_score=game.color_point, my_score=game.my_point if game.my_color == 'colors' else None)))

score_df = pd.DataFrame(scores).set_index(['date', 'game_num', 'team'], drop=False).sort_values('end_time')

In [43]:
score_df

color_wins       date            end_time  game_num  \
date       game_num team                                                        
2018-03-23 0        white          3 2018-03-23 2018-03-23 12:31:39         0   
                    color          3 2018-03-23 2018-03-23 12:31:39         0   
           1        white          3 2018-03-23 2018-03-23 12:46:51         1   
                    color          3 2018-03-23 2018-03-23 12:46:51         1   
           2        white          3 2018-03-23 2018-03-23 13:17:41         2   
                    color          3 2018-03-23 2018-03-23 13:17:41         2   

                          game_winner  my_score   team  team_score white_wins  
date       game_num team                                                       
2018-03-23 0        white       Color       NaN  white           3          0  
                    color       Color       3.0  color           5          0  
           1        white       Color       NaN  white           2          0  
                    color       Color       3.0  color           5          0  
           2        white       Color       NaN  white           4          0  
                    color       Color       1.0  color           5          0

In [44]:
out_df = score_df[['date', 'white_wins', 'color_wins', 'game_num', 'game_winner', 'team', 'team_score', 'my_score']].fillna(value='')
out_df['date'] = out_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [45]:
out_df

date white_wins color_wins  game_num  \
date       game_num team                                                
2018-03-23 0        white  2018-03-23          0          3         0   
                    color  2018-03-23          0          3         0   
           1        white  2018-03-23          0          3         1   
                    color  2018-03-23          0          3         1   
           2        white  2018-03-23          0          3         2   
                    color  2018-03-23          0          3         2   

                          game_winner   team  team_score my_score  
date       game_num team                                           
2018-03-23 0        white       Color  white           3           
                    color       Color  color           5        3  
           1        white       Color  white           2           
                    color       Color  color           5        3  
           2        white       Color  white           4           
                    color       Color  color           5        1

In [28]:
data = out_df.as_matrix().tolist()
wks = sh.worksheet_by_title('game_summaries')
wks.insert_rows(2, values=data, number=len(data))